In [1]:
%load_ext dotenv
%dotenv

In [2]:
from VLA_LLM import prompts
from VLA_LLM.agents import ZeroShotVLAAgent
from VLA_LLM.tools import AppointmentAvailabilityTool
from VLA_LLM.tools import AppointmentSchedulerTool
from VLA_LLM.tools import AppointmentSchedulerAndAvailabilityTool
from VLA_LLM.tools import CurrentTimeTool
from VLA_LLM.community_info import EXAMPLE_COMMUNITY_DATA
from VLA_LLM.community_info import community_dict_to_prompt

In [3]:
# most deterministic results
temperature = 0

In [4]:
community_info = community_dict_to_prompt(EXAMPLE_COMMUNITY_DATA)

In [5]:
print(community_info)

Parking options:
  1. Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available

Pet policy: Pets Allowed. Extra fee for pets: 50.0


In [6]:
# tools = [CurrentTimeTool(), AppointmentSchedulerAndAvailabilityTool()]
tools = [CurrentTimeTool(), AppointmentSchedulerTool(), AppointmentAvailabilityTool()]

In [7]:
prefix = prompts.prompt_three_tool_explicit.format(community_info=community_info)

In [8]:
agent = ZeroShotVLAAgent(temperature, prefix, tools)

### Test Conversation 1

Basic questions about the community.

In [9]:
agent.agent_chain.run(input="What is the parking situation?")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `
Thought: I need to provide information about the parking options.

Action: None
`

In [10]:
agent.agent_chain.run("i have a dog - can i bring him?")



> Entering new AgentExecutor chain...

Thought: I need to answer the prospect's question about the pet policy
Action: None
Final Answer: Yes, we do allow pets. There is an extra fee of $50.0 per pet.

> Finished chain.


'Yes, we do allow pets. There is an extra fee of $50.0 per pet.'

Multiple inquiries.

In [11]:
agent.agent_chain.run(input="What is the parking situation? can i bring my dog as well?")



> Entering new AgentExecutor chain...

Thought: I need to answer the parking situation and pet policy
Action: get_current_time
Action Input: N/A
Observation: Wednesday 2023-07-26 10:45:57
Thought: I need to answer the pet policy
Action: N/A
Action Input: N/A
Observation: N/A is not a valid tool, try another one.
Thought: I need to answer the pet policy
Action: N/A
Action Input: N/A
Observation: N/A is not a valid tool, try another one.
Thought: I need to answer the pet policy
Action: N/A
Action Input: Pets Allowed. Extra fee for pets: 50.0
Observation: N/A is not a valid tool, try another one.
Thought: I need to answer the parking situation
Action: N/A
Action Input: Parking option 1
     Parking type: Parking Available
     Parking fee: $100.0
     Parking fee term: Monthly
     Parking notes: noties parking available
Observation: N/A is not a valid tool, try another one.
Thought: I now know the answer to the questions
Final Answer: Yes, we do allow pets. There is an extra fee of $50

'Yes, we do allow pets. There is an extra fee of $50.0 per pet. Parking option 1 is available with a parking fee of $100.0 per month.'

### Test Conversation 2

Asking about an available appointment date.

In [12]:
agent.agent_chain.run(input="Can I schedule a tour for 7/27?")



> Entering new AgentExecutor chain...

Thought: I need to check the availability of appointments on 7/27
Action: appointment_availability
Action Input: 7/27
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought: I need to get the current time to convert the appointment date into the correct format
Action: get_current_time
Action Input: None
Observation: Wednesday 2023-07-26 10:46:13
Thought: I now have the current time and can convert the appointment date into the correct format
Action: appointment_availability
Action Input: 2023-07-27
Observation: Available appointment times on that date are 09:00:00, 10:00:00, 11:00:00. 
Thought: I now know the available appointment times
Final Answer: We have available appointment times on 7/27 at 9 AM, 10 AM, and 11 AM. Would you like to schedule a tour for one of those times?

> Finished chain.


'We have available appointment times on 7/27 at 9 AM, 10 AM, and 11 AM. Would you like to schedule a tour for one of those times?'

In [13]:
agent.agent_chain.run(input="Can I book one for 11am?")



> Entering new AgentExecutor chain...

Thought: I need to check if 11am is available
Action: appointment_availability
Action Input: 7/27
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought: I need to convert the date to the correct format
Action: get_current_time
Action Input: None
Observation: Wednesday 2023-07-26 10:46:26
Thought: I now have the current time and date
Action: appointment_availability
Action Input: 2023-07-27
Observation: Available appointment times on that date are 09:00:00, 10:00:00, 11:00:00. 
Thought: I now know the available appointment times
Final Answer: Yes, we have an available appointment time for 11 AM on 7/27. Would you like to book it?

> Finished chain.


'Yes, we have an available appointment time for 11 AM on 7/27. Would you like to book it?'

### Test Conversation 3

Asking about an unavailable appointment date.

In [14]:
agent.agent_chain.run(input="You do have any available appointments on 7/29?")



> Entering new AgentExecutor chain...

Thought: I need to check the availability of appointments on 7/29
Action: appointment_availability
Action Input: 7/29
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought: I need to convert the date into the correct format
Action: get_current_time
Action Input: None
Observation: Wednesday 2023-07-26 10:46:37
Thought: I now have the current time and date
Action: appointment_availability
Action Input: 2023-07-29
Observation: There are no available times on the requested date.
Thought: I now know the final answer
Final Answer: Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?

> Finished chain.


'Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?'

### Test Conversation 4

Asking about an available appointment time.

In [15]:
agent.agent_chain.run(input="Can I book a tour for 7/29 at 1pm?")



> Entering new AgentExecutor chain...

Thought: Prospect is asking for an appointment on 7/29 at 1pm
Action: get_current_time
Action Input: N/A
Observation: Wednesday 2023-07-26 10:46:49
Thought: I need to check if there are any available appointments on 7/29 at 1pm
Action: appointment_availability
Action Input: YYYY-MM-DD format of 7/29
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought: I need to convert the date into YYYY-MM-DD format
Action: get_current_time
Action Input: N/A
Observation: Wednesday 2023-07-26 10:46:52
Thought: I need to check if there are any available appointments on 7/29 at 1pm
Action: appointment_availability
Action Input: 2023-07-29
Observation: There are no available times on the requested date.
Thought: I now know the final answer
Final Answer: Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?

> Finished chain.


'Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?'

### Test Conversation 5

Asking about an available appointment time in relative terms.

In [16]:
agent.agent_chain.run(input="Can I book a tour for this Thursday at 1pm?")



> Entering new AgentExecutor chain...
Thought: I need to check if there are any available times on this Thursday
Action: appointment_availability
Action Input: current day of week + 1 week
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought: I need to convert the date into the correct format
Action: get_current_time
Action Input: None
Observation: Wednesday 2023-07-26 10:47:19
Thought: I now have the correct date format
Action: appointment_availability
Action Input: 2023-07-29
Observation: There are no available times on the requested date.
Thought: I now know the final answer
Final Answer: Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?

> Finished chain.


'Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?'

### Test Conversation 6

Asking about an unavailable appointment time.

In [17]:
agent.agent_chain.run(input="Can I book a tour for 7/30 at 3?")



> Entering new AgentExecutor chain...
Thought: I need to check if there are available times on 7/30
Action: appointment_availability
Action Input: 7/30
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought: I need to convert the date into the correct format
Action: get_current_time
Action Input: None
Observation: Wednesday 2023-07-26 10:47:41
Thought: I need to convert the date into the correct format
Action: appointment_availability
Action Input: 2023-07-30
Observation: There are no available times on the requested date.
Thought: I now know the final answer
Final Answer: Unfortunately, there are no available times on 7/30. Would you like to schedule a tour for another date?

> Finished chain.


'Unfortunately, there are no available times on 7/30. Would you like to schedule a tour for another date?'

### Test Conversation 7

Asking about an appointment and an inquiry.

In [18]:
agent.agent_chain.run(input="Can I book a tour for 7/29 at 1pm? i have a car - what is the parking situation like?")



> Entering new AgentExecutor chain...

Thought: I need to answer the parking question first
Action: get_current_time
Action Input: None
Observation: Wednesday 2023-07-26 10:47:55
Thought: I need to check the availability of the appointment
Action: appointment_availability
Action Input: 7/29
Observation: Appointment date is not in the correct format. The agent should provide it in YYYY-MM-DD format, then run again.
Thought: I need to convert the date to the correct format
Action: get_current_time
Action Input: None
Observation: Wednesday 2023-07-26 10:47:58
Thought: I need to check the availability of the appointment
Action: appointment_availability
Action Input: 2023-07-29
Observation: There are no available times on the requested date.
Thought: I now know the final answer
Final Answer: Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?

> Finished chain.


'Unfortunately, there are no available times on 7/29. Would you like to schedule a tour for another date?'